# Building Feature Based Grammar

In [1]:
import nltk

### Grammarical Features

In contrast to feature extractors, which record features that have been automatically detected, we are now going to declare the features of words and phrases. We start off with a very simple example, using dictionaries to store features and their values.

In [8]:
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF':'k'}
chase = {'CAT':'V', 'ORTH': 'chased', 'REL': 'chase'}

CAT - Grammatical Category<br>
ORTH - Orthography (spelling)<br>
The above are shared features<br>

Semantically-oriented feature: kim['**REF**'] is intended to give the referent of kim, while chase['**REL**'] gives the relation expressed by chase. In the context of rule-based grammars, such pairings of features and values are known as feature **structures**.

Feature structures include various kinds of information about grammatical entities. 

In the case of a verb, one must know what 'semantic role' is played by the arguments of the verb. For example, 'chased', the subject plays the role of 'agent' while object has the role of 'patient'. 

 Let's add this information, using 'sbj' and 'obj' as placeholders which will get filled once the verb combines with its grammatical arguments:

In [9]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

If we process 'Kim chased Lee', we want to bind the verb's agent role to the subject and the patient role to the 'object'. We do this by linking to the REF feature of the relevant NP.

In the following example, we make the simple-minded assumption that the NPs immediately to the left and right of the verb are the subject and object respectively. We also add a feature structure for Lee to complete the example.

In [10]:
sent = 'Kim chased Lee'
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs
        
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']

for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print('%-5s ==> %s' % (k, verb[k]))

ORTH  ==> chased
REL   ==> chase
AGT   ==> k
PAT   ==> l


The same approach could be adopted for a different verb, say surprise, though in this case, the subject would play the role of "source" (SRC) and the object, the role of "experiencer" (EXP):

In [11]:
surprise = {'CAT': 'V', 'ORTH': 'surprised', 'REL': 'surprise',
       'SRC': 'sbj', 'EXP': 'obj'}

But this type of construction of feature structures is ad hoc. We want something more generic. 

In [12]:
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


### Pocessing Feature Structures

Feature Structures in NLTK are declared with the FeatStruct() constructor. Atomic feature values can be strings or integers

In [13]:
fs1 = nltk.FeatStruct(TENSE = 'past', NUM = 'sg')
print(fs1)

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]


A feature structure is actually just a kind of dictionary, and so we access its values by indexing in the usual way. We can use our familiar syntax to assign values to features:

In [14]:
fs1 = nltk.FeatStruct(PER = 3, NUM = 'pl', GND= 'fm')
fs1['GND']

'fm'

In [15]:
fs1['CASE'] = 'acc'
print(fs1)

[ CASE = 'acc' ]
[ GND  = 'fm'  ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]


We can also define feature structures that have complex values, as discussed earlier

In [16]:
fs2 = nltk.FeatStruct(POS = 'N', AGR = fs1)
print(fs2)

[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fm'  ] ]
[       [ NUM  = 'pl'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]


In [18]:
fs2['AGR']

[CASE='acc', GND='fm', NUM='pl', PER=3]

An alternative method of specifying feature structures is to use a bracketed string consisting of feature-value pairs in the format feature=value, where values may themselves be feature structures

In [20]:
nltk.FeatStruct('[POS = "N", AGR = [PER =3, GND = "fem", NUM = "pl"]]')

[AGR=[GND='fem', NUM='pl', PER=3], POS='N']

Feature structures are not inherently tied to linguistic objects; they are general purpose structures for representing knowledge. For example, we could encode information about a person in a feature structure:

In [24]:
nltk.FeatStruct(NAME = 'Lee', TELNO= '012823', AGE = 33)

[AGE=33, NAME='Lee', TELNO='012823']

Feature Structures can be viewed as Directed Acyclic Graphs 

In order to indicate reentrancy in our matrix-style representations, we will prefix the first occurrence of a shared feature structure with an integer in parentheses, such as (1). Any later reference to that structure will use the notation ->(1), as shown below.



In [28]:
f1 = nltk.FeatStruct("""[NAME = 'LEE', ADDRESS =(1)[NUMBER = 74, 
                    STREET = 'RUE PASCAL'], SPOUSE = [NAME = 'KIM', 
                    ADDRESS -> (1)]]""")
print(f1)

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'RUE PASCAL' ] ]
[                                         ]
[ NAME    = 'LEE'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'KIM' ]           ]


In [29]:
f1['SPOUSE']['ADDRESS']

[NUMBER=74, STREET='RUE PASCAL']